## CVRP Package Demo

In this document we will demonstrate the usage of our project by:  
* solving a single instance of the CVRP 
* testing multiple optimization parameters and problems
* varying a single parameter to generate data

In [ ]:
from cvrp_class import CVRP
import pandas as pd
import datetime
import matplotlib.pyplot as plt

Below, we instantiate a CVRP object to solve the Georgia subproblem. We select the first DC, allow for math programming solutions of the vehicle subproblems, and set our distance matrix to be generated from data pulled from the Google Distance Matrix API. We then plot the solution using the built in method `get_vrp_plot()`

In [ ]:
model = CVRP(state = 'GA',
             dc_index = 0,
             vehicle_capacity = 30000,
             optimal_routing = True,
             google_dr_dist = True)

model.solve(method = 'clark-wright',
           verbose = True)
model.get_vrp_plot()

Below, we gather data on all solutions procedures discussed in our presentation. Runtime was aproximately 1 hr, with the ~ 50k required calls to the Google Distance Matrix API being the bottleneck.

In [ ]:
test_states = ['CA','TX','NY','GA','AL']

full_start = datetime.datetime.now()

results_list = []
for state in test_states:
    print(f'State: {state}')
    for google_bool in [True,False]:
        
        if state == 'CA' and google_bool == False:
            break
        if state == 'TX' and google_bool == False:
            break
        
        if state == 'CA' or state == 'TX':
            google_bool = False
            print(f'Changing {state} bool to false')
        test = CVRP(state,
                    dc_index = 0,
                    vehicle_capacity = 30000,
                    optimal_routing = False,
                    google_dr_dist = google_bool)
        print(f'Google Bool: {test.google_dr_dist}')
        for dc in [0,1]:
            test.dc_index = dc
            print(f'DC: {dc}')
            
            for alg in ['Sweep','Clark-Wright']:
                print(f'Method:{alg}')
                for routing in [False,True]:
                    print(f'Opt Routing: {routing}')
                    
                    test.optimal_routing = routing
                    start_time = datetime.datetime.now()
                
                    test.solve(alg,verbose = False)
                    
                    runtime = datetime.datetime.now() - start_time
                    runtime = runtime.total_seconds()
                    
                    temp_list = []
                    temp_list.append(state)
                    temp_list.append(test.current_dc)
                    temp_list.append(google_bool)
                    temp_list.append(alg)
                    temp_list.append(routing)
                    temp_list.append(test.num_vehicles)
                    temp_list.append(test.total_time)
                    temp_list.append(test.total_cost)
                    temp_list.append(runtime)
                    
                    results_list.append(temp_list)
                
                
df= pd.DataFrame(results_list)
    
#df = df.transpose()
df.columns = ['State',
              'DC',
              'Real Distances',
              'Method',
              'Optimal Vehicle Routing',
              '# of Vehicles',
              'Total Time',
              'Total Cost',
              'Runtime']
    
df['Runtime'] = df['Runtime'].apply('{:,.2f}'.format)
df['Total Cost'] = df['Total Cost'].apply('{:,.2f}'.format)
df['Total Time'] = df['Total Time'].apply('{:,.2f}'.format)
df = df.sort_values(['State','Total Cost'])

print(df)

df.to_csv('all_results.csv')

print(datetime.datetime.now()-full_start)

Below, we investigate the the effect of capacity on the best-tested soltuion precedure in Alabama.

In [ ]:
model = CVRP('AL',
             dc_index = 1,
             vehicle_capacity = 10000,
             optimal_routing = True,
             google_dr_dist = True)

data_dict = {}

for capacity in range(10000,51000,1000):
    model.vehicle_capacity = capacity
    model.solve('clark-wright',verbose = True)
    data_dict[capacity] = model.total_cost



df = pd.DataFrame.from_dict(data_dict,orient='index')

df.to_csv('capacity_investigation_alabama_raw_data.csv')

Below, we plot the data found in the capacity experiment.

In [ ]:
df = pd.read_csv('capacity_investigation_alabama_raw_data.csv')

df.columns = ['Vehicle Capacity','Total Cost']

df = df.set_index('Vehicle Capacity')

plt.style.use('ggplot')
fig, ax = plt.subplots(1,1, figsize=(12, 9))

ax.title.set_text('Total Cost vs. Vehicle Capacity')
ax.plot(df)
ax.set_xlabel('Vehicle Capacity')
ax.set_ylabel('Total Cost $')
plt.show()